# Python и интернет. Модуль requests

**План**:

1. Запросы
2. Requests
4. bs4
5. Задание на семинар

## Как выкачать интернет
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек: **requests** и **urllib**.

## Requests

Можно почитать в [документации](https://requests.readthedocs.io/en/latest/) или [тут](https://realpython.com/python-requests/).

Допустим, мы хотим скачать главную страницу Хабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [9]:
import requests

In [129]:
response = requests.get("https://habr.com/ru/")

В response теперь лежит отет сервера. Это не просто HTML-код страницы, а еще дополнительная информация. Если мы просто выведем этот `response`, нам покажется только код (если статус $-$ 200, то все ок).

In [14]:
response

<Response [200]>

In [16]:
response.status_code

200

А вот в атрибуте `text` уже лежит HTML-код

In [19]:
print(response.text[:700])

<!DOCTYPE html>
<html lang="ru">

  <head>
    <title>Публикации &#x2F; Моя лента &#x2F; Хабр</title>
<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">
<link href="https://habr.com/ru/feed/" rel="canonical" data-hid="e3fa780">
<meta itemprop="image" content="/img/habr_ru.png">
<meta property="og:image" content="/img/habr_ru.png">
<meta property="og:image:width" content="1200">
<meta property="og:image:height" content="630">
<meta property="aiturec:image" content="/img/habr_ru.png">
<meta name="twitter:image" content="/img/habr_ru.png">
<meta property="vk:image" content="/img/habr_ru.png?format=vk">
<meta property="fb:app_id" content="444736788986613">
<meta property="fb:pages


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу в питоне, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5".
Но можно, например, представиться Мозиллой:

In [22]:
url = 'https://habr.com/ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get("https://habr.com/ru/", headers={'User-Agent': user_agent})

Или использовать специальную библиотеку

In [29]:
# !pip install fake_useragent --q

In [25]:
from fake_useragent import UserAgent

In [27]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Agency/98.8.8175.80'

In [31]:
response = requests.get("https://habr.com/ru/", headers={'User-Agent': user_agent})

Можно посмотреть, что еще можно достать из response.

Функция ```dir``` выдает список методов и параметров объекта.

In [33]:
[i for i in dir(response) if not i.startswith("_")]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

- Кодировка:

In [36]:
response.encoding

'utf-8'

- Заголовки (техническая информация):

In [39]:
dict(response.headers)

{'Server': 'QRATOR',
 'Date': 'Fri, 08 Nov 2024 08:36:52 GMT',
 'Content-Type': 'text/html; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Keep-Alive': 'timeout=15',
 'Vary': 'Accept-Encoding, Accept-Encoding',
 'X-DNS-Prefetch-Control': 'off',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Download-Options': 'noopen',
 'X-Content-Type-Options': 'nosniff',
 'X-XSS-Protection': '1; mode=block',
 'ETag': 'W/"459a1-7M0fPtdI9IlGSsTY1Pt2F0Zht+k"',
 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains',
 'X-Request-Id': '3a59d0f8cd5dd62a957fbc3896d0f35f',
 'X-Request-Geoip-Country-Code': 'RU',
 'X-Request-Detected-Device': 'desktop',
 'Content-Encoding': 'gzip'}

- Адрес запроса:

In [42]:
response.url

'https://habr.com/ru/feed/'

* Содержимое страницы:

In [45]:
print(response.text[:300])

<!DOCTYPE html>
<html lang="ru">

  <head>
    <title>Публикации &#x2F; Моя лента &#x2F; Хабр</title>
<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">
<link href="https://habr.com/ru/feed/" rel="canonical" data-hid="e3fa780">
<meta itemprop="image" content="/img/habr_ru.png">
<meta 


Ура, всё на месте!

Но что всё это значит? Что такое HTML и как вообще из него доставать какую-то информацию?

Ответ: по **тегам**! Например, в куске HTML сверху есть теги `<title> </title>` (теги всегда обрамляют с двух сторон то, что находится под этим тегом). В `<title>` в данном случае лежит заголовок этой интернет-страницы.

Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

1. регулярные выражения ([плохой вариант](https://stackoverflow.com/questions/590747/using-regular-expressions-to-parse-html-why-not))
2. специальные библиотеки питона, например, BeautifulSoup (bs4) или [lxml](https://lxml.de/) (хороший вариант)

## BeautifulSoup

[Документация](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

Код страницы парсится как иерархия тегов (как они есть в html коде, один вложен в другой) и можно искать элементы с помощью разных методов, использовать атрибуты и т.д.

In [49]:
from bs4 import BeautifulSoup

Сначала инициализируем объект `BeautifulSoup`. Потом применим метод `find` и в скобочках укажем теги, по которым ищем. У некоторых тегов в HTML (как и в нашем случае) бывает еще и `class` и какие-нибудь еще атрибуты. Такие вещи мы задаем словариком.

Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем HTML файле.

In [131]:
soup = BeautifulSoup(response.text, 'html.parser')  # инициализируем (создаем) soup

post = soup.find('h2', {'class': 'tm-title tm-title_h2'})
print(post.get_text(), end="\n\n\n")
print(post.prettify())

Как использовать плагины SBP для развития b2b продукта


<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/857090/">
  <span>
   Как использовать плагины SBP для развития b2b продукта
  </span>
 </a>
 <!--]-->
</h2>



Но мы хотим получить все заголовки постов! Метод `find_all` возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [133]:
for post in soup.find_all('h2', {'class': 'tm-title tm-title_h2'})[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- ' * 10)  # для красоты

Как использовать плагины SBP для развития b2b продукта
<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/857090/">
  <span>
   Как использовать плагины SBP для развития b2b продукта
  </span>
 </a>
 <!--]-->
</h2>

-- -- -- -- -- -- -- -- -- -- 
Я бы строить дом пошел или на какую профессию сменить ИТ
<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/857088/">
  <span>
   Я бы строить дом пошел или на какую профессию сменить ИТ
  </span>
 </a>
 <!--]-->
</h2>

-- -- -- -- -- -- -- -- -- -- 
Как мы в билайне боремся со спам-звонками с помощью машинного обучения, часть 1
<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-

## Задание на семинар 1

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там проиcходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в HTML)

```
<article class="tm-articles-list__item" data-navigatable="" id="771476" tabindex="0">
 <div class="tm-article-snippet tm-article-snippet">
  <div class="tm-article-snippet__meta-container">
   <div class="tm-article-snippet__meta">
    <span class="tm-user-info tm-article-snippet__author">
     <a class="tm-user-info__userpic" href="/ru/users/darinka666/" title="darinka666">
      <div class="tm-entity-image">
       <img alt="" class="tm-entity-image__pic" height="24" src="https://assets.habr.com/habr-web/img/avatars/093.png" width="24"/>
      </div>
     </a>
     <span class="tm-user-info__user tm-user-info__user_appearance-default">
      <a class="tm-user-info__username" href="/ru/users/darinka666/">
       darinka666
       <!-- -->
      </a>
      <span class="tm-article-datetime-published">
       <time datetime="2023-11-02T09:22:25.000Z" title="2023-11-02, 12:22">
        11 минут назад
       </time>
      </span>
     </span>
    </span>
   </div>
   <!-- -->
  </div>
  <h2 class="tm-title tm-title_h2">
   <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/mts_ai/articles/771476/">
    <span>
     Обзор Llemma: новая математическая open-source модель
    </span>
   </a>
  </h2>
  <div class="tm-article-snippet__stats">
   <div class="tm-article-complexity tm-article-complexity_complexity-medium">
    <span class="tm-svg-icon__wrapper tm-article-complexity__icon">
     <svg class="tm-svg-img tm-svg-icon" height="24" width="24">
      <title>
       Уровень сложности
      </title>
      <use xlink:href="/img/megazord-v28.2fb1b1c1..svg#complexity-medium">
      </use>
     </svg>
    </span>
    <span class="tm-article-complexity__label">
     Средний
    </span>
   </div>
```
(и так далее; часть вывода обрезана: обратите внимание, нет закрывающего тега `</article>`)

In [90]:
for post in soup.find_all("article")[:3]:
    print("TEXT:")
    print(post.get_text()[:500])
    print('-- '*10)
    print("HTML:")
    print(post.prettify()[:500])

    print('-- '*10)
    print('-- '*10)
    print('-- '*10)

TEXT:
Статьяvolgeorge 4 минуты назадТоп-6 пиар-агентств для работы в России: личный рейтинг пиарщика компанииУровень сложностиПростойВремя на прочтение5 минКоличество просмотров7ОбзорРейтинг0Добавить в закладки0Комментарии0
-- -- -- -- -- -- -- -- -- -- 
HTML:
<article class="tm-articles-list__item" data-navigatable="" data-test-id="articles-list-item" id="856926" tabindex="0">
 <div class="tm-article-snippet tm-article-snippet">
  <!--[-->
  <div class="publication-type-label publication-type-label_type-article">
   <span class="publication-type-label__label publication-type-label__label_type-article">
    Статья
   </span>
  </div>
  <!--]-->
  <div class="tm-article-snippet__meta-container">
   <div class="tm-article-snippet__meta">
    <span class
-- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- 
TEXT:
Постnastyakopi 11 минут назадКоличество просмотров368Как у вас с пространственным мышлением? Проверьте себя — решите задачуУсловиеАйтишник-

### Решение задания на семинар 1

In [168]:
try:  # чтобы исключить случаи, когда нет юзернейма / времени / заголовка
    for post in soup.find_all("article"):
        user_name = post.find('a', {'class': 'tm-user-info__username'}).get_text()
        time_article = post.find('time').get_text()
        header_article = post.find('h2', {'class': 'tm-title tm-title_h2'}).get_text()
        print('Name:', user_name)
        print('Time:', time_article)
        print('Header:', header_article, end='\n\n')
except AttributeError:
    # ошибка AttributeError возникает, когда от объекта типа None вызывают метод .get_text()
    pass

Name: Caterin 
Time: 1 минуту назад
Header: Как использовать плагины SBP для развития b2b продукта

Name: HunterXXI 
Time: 4 минуты назад
Header: Я бы строить дом пошел или на какую профессию сменить ИТ

Name: Beeline_tech 
Time: 27 минут назад
Header: Как мы в билайне боремся со спам-звонками с помощью машинного обучения, часть 1

Name: A1LEks 
Time: 28 минут назад
Header: Как мы разработали и добавили «истории» в сервис для поиска работы за 3 месяца

Name: Oksana_Nedvigina 
Time: 29 минут назад
Header: Чтение на выходные: «Великий Китайский Файрвол. Как создать свой собственный интернет и управлять им» Джеймса Гриффитса

Name: Oksana_Nedvigina 
Time: 30 минут назад
Header: Чтение на выходные: «Великий Китайский Файрвол. Как создать свой собственный интернет и управлять им» Джеймса Гриффитса

Name: ptsecurity 
Time: 37 минут назад
Header: Вести с полей киберинцидентов: команда расследователей делится итогами проектов 2023-2024

Name: k0mar0v 
Time: 43 минуты назад
Header: От мышей на 

## Задание на семинар 2

Скачать главную страницу Яндекс.Погоды и
    
- распечатать сегодняшнюю температуру и облачность
- распечатать время восхода и заката
- погоду на завтра

Простая версия: вместо Яндекс.Погоды возьмите [этот](https://simple-weather-website.netlify.app/) сайт и получите температуру, влажность и скорость ветра.

## [Хорошая статья про это все](https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/)

# Бонус

Иногда можно не изгаляться с HTML, а воспользоваться специальным инструментом: **API**.

API расшифровывается как Application Programming Interface. Это набор правил, которыми одна программа может общаться и взаимодействовать с другой, в том числе с сайтами. Например, "сделай запрос к такой-то странице в таком-то формате, чтобы получить JSON с данными о погоде и не парсить HTML руками".

Но сначала пример попроще. Это сайт, который просто возвращает случайные картинки с лисами (есть [агрегатор](https://publicapi.dev/) разных API, там много такого можно найти):

In [61]:
fox = requests.get("https://randomfox.ca/floof/")

Обратите внимание, что `.text` вернет нам *строку*:

In [63]:
fox.text

'{"image":"https:\\/\\/randomfox.ca\\/images\\/12.jpg","link":"https:\\/\\/randomfox.ca\\/?i=12"}'

Но у респонза есть еще метод `.json()`. В данном случае он сразу приведет к списку или словарю. В данном случае словарь, так как API возвращает словарь с несколькими ключами.



In [66]:
fox.json()

{'image': 'https://randomfox.ca/images/12.jpg',
 'link': 'https://randomfox.ca/?i=12'}

Более сложный, но и более интересный пример: https://newsapi.org/

 На главной нужно получить ключ. Многие API доступны только по ключам: например, API гитхаба так определяет, к каким репозиториям у вас есть доступ, а к каким нет; некоторые API ограничивают число запросов на одного пользователя; и т.д.

 **Ключ $-$ это личная информация**, как, например, пароль. Ключ нежелательно вставлять в код, потому что будет легко забыть его удалить перед тем, как вы где-то этот код опубликуете. Лучше положить в отдельный файл и его читать:

In [68]:
with open("news_api_key.txt", "r") as f:
  api_key = f.read()

А если вы работаете в колабе, то можно положить ключ в секреты колаба и сделать вот так:

In [ ]:
from google.colab import userdata

api_key = userdata.get('NEWS_API_KEY')

Какие параметры можно указывать в запросе, смотрите [здесь](https://newsapi.org/docs/endpoints/everything).

In [70]:
news = requests.get(
    f"https://newsapi.org/v2/everything?q=Apple&from=2024-10-25&sortBy=popularity&apiKey={api_key}"
    ).json()

In [74]:
news

{'status': 'ok',
 'totalResults': 20309,
 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'},
   'author': 'Will Shanklin',
   'title': 'Apple Intelligence is coming to EU iPhones and iPads in April',
   'description': 'Folks in the European Union finally know when Apple Intelligence will be available on their mobile devices. Apple told EU users on Monday that its AI suite will arrive in April 2025. The first Apple Intelligence features, including Writing Tools and AI notifi…',
   'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_9e418204-613e-41ea-849e-1acac15729e7',
   'urlToImage': None,
   'publishedAt': '2024-10-28T19:10:28Z',
   'content': "If you click 'Accept all', we and our partners, including 237 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"},
  {'source': {'id': None, 'name': 'Yahoo Entertainment'},
   'author': 'Lawrence Bonk'

In [76]:
news["totalResults"]

20309

In [78]:
articles = news["articles"][:5]

In [80]:
articles

[{'source': {'id': None, 'name': 'Yahoo Entertainment'},
  'author': 'Will Shanklin',
  'title': 'Apple Intelligence is coming to EU iPhones and iPads in April',
  'description': 'Folks in the European Union finally know when Apple Intelligence will be available on their mobile devices. Apple told EU users on Monday that its AI suite will arrive in April 2025. The first Apple Intelligence features, including Writing Tools and AI notifi…',
  'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_9e418204-613e-41ea-849e-1acac15729e7',
  'urlToImage': None,
  'publishedAt': '2024-10-28T19:10:28Z',
  'content': "If you click 'Accept all', we and our partners, including 237 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"},
 {'source': {'id': None, 'name': 'Yahoo Entertainment'},
  'author': 'Lawrence Bonk',
  'title': 'iOS 18.2 will include daily Sudoku puzzles for Ap

Другие API:

1. [OpenWeatherMap](https://openweathermap.org/api) $-$ предоставляет информацию о погоде, прогнозы и исторические данные по всему миру. Есть бесплатный лимит запросов, но все равно требуется ввести данные банковской карты.

2. [GitHub](https://developer.github.com/v3/) $-$ API для доступа к данным о репозиториях, пользователях, коммитах и т.д.

3. [Open Trivia API](https://opentdb.com/api_config.php) $-$ API для получения вопросов и ответов из базы данных триивиума (викторины).

In [82]:
requests.get("https://opentdb.com/api.php?amount=10").json()

{'response_code': 0,
 'results': [{'type': 'multiple',
   'difficulty': 'hard',
   'category': 'Science: Computers',
   'question': 'Who is the original author of the realtime physics engine called PhysX?',
   'correct_answer': 'NovodeX',
   'incorrect_answers': ['Ageia', 'Nvidia', 'AMD']},
  {'type': 'multiple',
   'difficulty': 'medium',
   'category': 'Entertainment: Music',
   'question': 'Which Beatle led the way across the zebra crossing on the Abbey Road album cover?',
   'correct_answer': 'John',
   'incorrect_answers': ['Paul', 'George', 'Ringo']},
  {'type': 'multiple',
   'difficulty': 'medium',
   'category': 'Celebrities',
   'question': 'What is Doug Walker&#039;s YouTube name?',
   'correct_answer': 'The Nostalgia Critic',
   'incorrect_answers': ['The Angry Video Game Nerd',
    'AngryJoeShow',
    'The Cinema Snob']},
  {'type': 'multiple',
   'difficulty': 'easy',
   'category': 'Entertainment: Cartoon &amp; Animations',
   'question': 'In The Simpsons, which war did 

А еще есть Drama Corpus, который делали в том числе при участии Школы Лингвистики: https://dracor.org/doc/api

In [84]:
requests.get("https://dracor.org/api/corpora/rus/play/gogol-revizor/cast").json()[:5]

[{'id': 'gorodnichij',
  'name': 'Городничий',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 21,
  'numOfSpeechActs': 172,
  'numOfWords': 4991,
  'degree': 26,
  'weightedDegree': 84,
  'closeness': 0.8760416666666667,
  'betweenness': 0.1264602852635307,
  'eigenvector': 0.24182361124623364},
 {'id': 'ammos_fedorovich_ljapkin_tjapkin',
  'name': 'Аммос Федорович',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 8,
  'numOfSpeechActs': 49,
  'numOfWords': 748,
  'degree': 21,
  'weightedDegree': 54,
  'closeness': 0.7377192982456141,
  'betweenness': 0.010129431082777935,
  'eigenvector': 0.23046261804194926},
 {'id': 'artemij_filippovich_zemljanika',
  'name': 'Артемий Филиппович Земляника',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 10,
  'numOfSpeechActs': 51,
  'numOfWords': 737,
  'degree': 21,
  'weightedDegree': 63,
  'closeness': 0.7377192982456141,
  'betweenness': 0.010129431082777935,
  'eigenvector': 0.23046261804194926},
 {'id': 'luka_luk